Giulia Midulla - 23330406

#Preparatory steps

In [1]:
# The %pip commands below are for use if you have to install the packages
# you can comment them out once you have installed them.
# %pip install nltk
# %pip install gensim
import glob # string manipulation for constructing directory paths
import nltk # bring in the Natural Language Tool Kit
import gensim # bring in Gensim
import os # handle Operating System file tasks
import os.path # Used to determine if a file or directory exists
import numpy as np # convenient mathematical handling
from random import shuffle # facility to generate random selections
from nltk.tokenize import TreebankWordTokenizer # String tokenizer
from gensim import models
from gensim.models.keyedvectors import KeyedVectors # word vector mapping
# Keras network construction and handling libraries
from keras.preprocessing import sequence
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, SimpleRNN, Conv1D, GlobalMaxPooling1D
print("Necessary packages installed....")

Necessary packages installed....


In [2]:
from google.colab import drive
drive.mount('/content/drive')
print("Google drive mounted....")

working_directory = '/content/drive/MyDrive/Colab Notebooks/Sem 2/Section 2'

if not os.path.exists(working_directory) :
  os.mkdir(working_directory)
  print("Created working directory....")

os.chdir(working_directory)
print("Set current directory to working directory.... :", working_directory)

Mounted at /content/drive
Google drive mounted....
Set current directory to working directory.... : /content/drive/MyDrive/Colab Notebooks/Sem 2/Section 2


In [3]:
print("You will need to put the two downloaded compressed files into your working directory", working_directory)

You will need to put the two downloaded compressed files into your working directory /content/drive/MyDrive/Colab Notebooks/Sem 2/Section 2


In [4]:
# Extract the Stanford Sentiment Analysis files if you haven't already
os.chdir(working_directory)
print("Set current directory to working directory.... :", working_directory)

if os.path.exists('aclImdb_v1.tar.gz') :
  !gunzip 'aclImdb_v1.tar.gz'

if os.path.exists('aclImdb_v1.tar') :
  !tar -xvf 'aclImdb_v1.tar'

# Extract the Google News trained word vectors if you haven't already
if os.path.exists('GoogleNews-vectors-negative300.bin.gz') :
  !gunzip 'GoogleNews-vectors-negative300.bin.gz'

#
# If you need to save space on your Google Drive, then you can uncomment the
# code below and delete the now unneeded compressed files
#
if os.path.exists(os.path.join(working_directory, 'aclImdb_v1.tar')) :
   print("Removing Stanford directory tarball to save space....")
   os.remove(os.path.join(working_directory, 'aclImdb_v1.tar'))
   print("Removed")

# Check directory contents
print("The files in my working directory are", os.listdir(working_directory))

Set current directory to working directory.... : /content/drive/MyDrive/Colab Notebooks/Sem 2/Section 2
The files in my working directory are ['GoogleNews-vectors-negative300.bin', 'aclImdb', 'prova.gdoc', 'prova cartella', 'cnn1_model.json', 'cnn1_weights.h5', 'model.png']


#General Assignment Tasks

In [5]:
def pre_process_data(filepath):
    """
    This is fairly generic code for sentiment analysis cleaning but
    it comes down to splitting the data into positive and negative
    sentiments and labelling it accordingly with target values for
    sentiment, 1 for positive and 0 for negative
    """
    positive_path = os.path.join(filepath, 'pos')
    negative_path = os.path.join(filepath, 'neg')
    pos_label = 1
    neg_label = 0
    dataset = []

    for filename in glob.glob(os.path.join(positive_path, '*.txt')):
        with open(filename, 'r') as f:
            dataset.append((pos_label, f.read()))

    for filename in glob.glob(os.path.join(negative_path, '*.txt')):
        with open(filename, 'r') as f:
            dataset.append((neg_label,f.read()))

    shuffle(dataset)

    return(dataset)

In [6]:
#os.chdir(os.path.join(working_directory,"aclImdb"))
#print("Acting on data in",os.path.join(working_directory,'aclImdb') )
dataset = pre_process_data(os.path.join(working_directory,"aclImdb",'train'))
# Have a quick look at a sample
dataset[42]

(0,
 "I had hoped this movie was going to be mildly entertaining, like other sorts of its genre. However, it was lame and I didn't find myself laughing very much. Watch it on HBO, maybe, or if you've got a free rental to waste and you need a movie to pass the time. But I don't recommend paying to see it.<br /><br />The plot is simple and straightforward, and it could have been funny, maybe, if the script was better. Jason Lee can be hilarious, and he gets a few laughs here and there, but the movie falls flat. Just don't go see this one. The directing is lackluster, but for what it is, directing isn't that important. I guess its main drawback is that it is just not very funny. See something else, don't waste your time here.")

In [7]:
w = models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', limit = 20000, binary=True)

In [8]:
def tokenize_and_vectorize(dataset):
    tokenizer = TreebankWordTokenizer()
    vectorized_data = []
    expected = []
    for sample in dataset:
        tokens = tokenizer.tokenize(sample[1])
        sample_vecs = []
        for token in tokens:
            try:
                sample_vecs.append(w[token])

            except KeyError:
                pass # No matching token in the downloaded Google word2vec vocab

        vectorized_data.append(sample_vecs)

    return vectorized_data

In [9]:
def collect_expected(dataset):
    """ Extract the target sentiments from the dataset """
    expected = []
    for sample in dataset:
        expected.append(sample[0])
    return expected

In [10]:
# Pad or truncate each sequence of input tokens in each review so that we have
# a fixed input length (**maxlen**) of tokens for network input

def pad_trunc(data, maximumlen):
    """
    Pad or truncate each review to the size set by the hyperparameter maxlen
    because we need all the inputs of the CNN to be of fixed size.
    """
    new_data = []

    zero_vector = []
    for _ in range(len(data[0][0])):
        zero_vector.append(0.0)

    for sample in data:
        if len(sample) > maximumlen: # if the input is too large, truncate it
            temp = sample[:maximumlen]
        elif len(sample) < maximumlen: # if the input is too small, pad it
            temp = sample
            # Append the appropriate number zero vectors to the list
            additional_elems = maximumlen - len(sample)
            for _ in range(additional_elems):
                temp.append(zero_vector)
        else:
            temp = sample
        new_data.append(temp)
    return new_data

In [11]:
vectorized_data = tokenize_and_vectorize(dataset)
expected = collect_expected(dataset)
print("Number of vectorized training data -", len(vectorized_data))

Number of vectorized training data - 4565


In [12]:
# When we no longer need the dataset, we delete it
# This is both good housekeeping and it stops
# Colab from complaining about RAM or disk overrun
del(dataset)

In [13]:
split_point = int(len(vectorized_data)*.8)
x_train = vectorized_data[:split_point]
y_train = expected[:split_point]
x_test = vectorized_data[split_point:]
y_test = expected[split_point:]

print("Number of training vectors - ", len(x_train))

Number of training vectors -  3652


In [14]:
del(vectorized_data) # Clear some more storage!
print("Cleared out unneeded data")

Cleared out unneeded data


In [15]:
maxlen = 100
batch_size = 32
embedding_dims = 300 # This is fixed as Google News used 300 dimensional vectors
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

In [16]:
# Pad or truncate the reviews and convert the inputs to an optimized
# Numpy format

x_train = pad_trunc(x_train, maxlen)
x_test = pad_trunc(x_test, maxlen)   # Does the same with the test cases

x_train = np.reshape(x_train, (len(x_train), maxlen, embedding_dims))
y_train = np.array(y_train)

x_test = np.reshape(x_test, (len(x_test), maxlen, embedding_dims))
y_test = np.array(y_test)

In [17]:
# Let people know what you are doing
print('Build CNN model for our MN5002 Advanced NLP course....')
model_cnn1 = Sequential()
model_cnn1.add(Conv1D(
    filters,
    kernel_size,
    padding='valid',
    activation='relu',
    strides=1,
    input_shape=(maxlen, embedding_dims))
)
model_cnn1.add(GlobalMaxPooling1D())
model_cnn1.add(Dense(hidden_dims)) # This is provides a fully connected hidden layer
model_cnn1.add(Dropout(0.2)) # A dropout of 0.2 is added here
model_cnn1.add(Activation('relu')) # The Rectified Linear Unit activation function is used here
model_cnn1.add(Dense(1)) # Add 1 dense layer
model_cnn1.add(Activation('sigmoid')) # Apply a sigmoid to the output of that layer to take the output to between 0 and 1
print('Model built')
model_cnn1.summary()
model_cnn1.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print('Model compiled')

Build CNN model for our MN5002 Advanced NLP course....
Model built
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 98, 250)           225250    
                                                                 
 global_max_pooling1d (Glob  (None, 250)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 250)               62750     
                                                                 
 dropout (Dropout)           (None, 250)               0         
                                                                 
 activation (Activation)     (None, 250)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 

In [18]:
model_cnn1.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test))

Epoch 1/2
115/115 [==============================] - 6s 11ms/step - loss: 0.0207 - accuracy: 0.9904 - val_loss: 1.7721e-06 - val_accuracy: 1.0000
Epoch 2/2
115/115 [==============================] - 1s 6ms/step - loss: 9.8392e-07 - accuracy: 1.0000 - val_loss: 2.0693e-07 - val_accuracy: 1.0000


In [19]:
os.chdir(working_directory)
model_cnn1_structure = model_cnn1.to_json()
with open("cnn1_model.json", "w") as json_file:
    json_file.write(model_cnn1_structure)
model_cnn1.save_weights("cnn1_weights.h5")

In [20]:
# Import the module to read in a JSON format model
# Remove the lines below with set of 3 inverted commas to enable code


from keras.models import model_from_json

# Now instantiate a model

os.chdir(working_directory)

with open("cnn1_model.json", "r") as json_file:
  json_string = json_file.read()
model_cnn1 = model_from_json(json_string)

# Once the model structure exists, set its characteristic weights

model_cnn1.load_weights('cnn1_weights.h5')
model_cnn1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 98, 250)           225250    
                                                                 
 global_max_pooling1d (Glob  (None, 250)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 250)               62750     
                                                                 
 dropout (Dropout)           (None, 250)               0         
                                                                 
 activation (Activation)     (None, 250)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 251       
                                                        

#Main Task 1

In [21]:
test1 = 'this movie is terrible'
print(test1, '- straightforward negative sentiment, used to check if the model works correctly\n')
test2 = 'this movie is wonderful, the best thing I\'ve ever seen I believe it to be an undisputed masterpiece'
print(test2, '- straightforward positive sentiment, used to check if the model works correctly\n')
test3 = 'this isn\'t the best movie of all time'
print(test3, '- negation of a positive sentiment, used to check the model\'s reaction\n')
test4 = 'this movie is as good as The Room'
print(test4, '- \"The Room\" is considered to be one of the worst movies ever made. This test is a trick\n')
test5 = 'how this movie is considered good will never fail to surprise me'
print(test5, '- different type of trick: the reviewer is admitting that the movie is generally considered good, but they have an opposing opinion\n')
test6 = 'watching this movie will make you feel like holding a puppy or eating chocolate'
print(test6, '- not explicit positive review\n')
test7 = 'Parasite is best described as a melancholy ghost story, albeit one disguised beneath umpteen layers of superbly designed (and impeccably photographed) generic mutations. Thrillingly played by a flawless ensemble cast who hit every note and harmonic resonance of Bong and co-writer Han Jin-won’s multitonal script, it’s a tragicomic masterclass that will get under your skin and eat away at your cinematic soul.'
print(test7, '- long, glowing review (source: The Guardian)')


this movie is terrible - straightforward negative sentiment, used to check if the model works correctly

this movie is wonderful, the best thing I've ever seen I believe it to be an undisputed masterpiece - straightforward positive sentiment, used to check if the model works correctly

this isn't the best movie of all time - negation of a positive sentiment, used to check the model's reaction

this movie is as good as The Room - "The Room" is considered to be one of the worst movies ever made. This test is a trick

how this movie is considered good will never fail to surprise me - different type of trick: the reviewer is admitting that the movie is generally considered good, but they have an opposing opinion

watching this movie will make you feel like holding a puppy or eating chocolate - not explicit positive review

Parasite is best described as a melancholy ghost story, albeit one disguised beneath umpteen layers of superbly designed (and impeccably photographed) generic mutations.

#Main Task 2

##Test 1

In the first test I'll use the preset hyperparameters:
```
maxlen = 100
batch_size = 32
embedding_dims = 300 # This is fixed as Google News used 300 dimensional vectors
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2
```


In [22]:
# Make a sample
sample_2 ="This is one of the worst movies I have ever seen. I am surprised anyone has recommended it, it is as bad as they get."

vec_list = tokenize_and_vectorize([(1, sample_2)]) # Feed a sample to the tokenizer and vectorizer

# Convert the input to a fixed input length by padding or truncating
test_vec_list = pad_trunc(vec_list, maxlen) # convert the input to a fixed input length

# Input it to the model to predict the sentiment for it
test_vec = np.reshape(test_vec_list, (len(test_vec_list), maxlen, embedding_dims))

# Present the prediction
model_cnn1.predict(test_vec) # predict the sentiment of the input


1/1 [==============================] - 0s 155ms/step


array([[1.1847078e-06]], dtype=float32)

Negative - OK

In [23]:
# Tokenize and vectorize it
vec_list = tokenize_and_vectorize([(1, test1)]) # Feed a sample to the tokenizer and vectorizer

# Convert the input to a fixed input length by padding or truncating
test_vec_list = pad_trunc(vec_list, maxlen) # convert the input to a fixed input length

# Input it to the model to predict the sentiment for it
test_vec = np.reshape(test_vec_list, (len(test_vec_list), maxlen, embedding_dims))

# Present the prediction
model_cnn1.predict(test_vec) # predict the sentiment of the input

1/1 [==============================] - 0s 20ms/step


array([[3.237688e-05]], dtype=float32)

Negative - OK

In [24]:
# Tokenize and vectorize it
vec_list = tokenize_and_vectorize([(1, test2)]) # Feed a sample to the tokenizer and vectorizer

# Convert the input to a fixed input length by padding or truncating
test_vec_list = pad_trunc(vec_list, maxlen) # convert the input to a fixed input length

# Input it to the model to predict the sentiment for it
test_vec = np.reshape(test_vec_list, (len(test_vec_list), maxlen, embedding_dims))

# Present the prediction
model_cnn1.predict(test_vec) # predict the sentiment of the input

1/1 [==============================] - 0s 18ms/step


array([[1.1365333e-06]], dtype=float32)

Positive - NO

In [25]:
# Tokenize and vectorize it
vec_list = tokenize_and_vectorize([(1, test3)]) # Feed a sample to the tokenizer and vectorizer

# Convert the input to a fixed input length by padding or truncating
test_vec_list = pad_trunc(vec_list, maxlen) # convert the input to a fixed input length

# Input it to the model to predict the sentiment for it
test_vec = np.reshape(test_vec_list, (len(test_vec_list), maxlen, embedding_dims))

# Present the prediction
model_cnn1.predict(test_vec) # predict the sentiment of the input

1/1 [==============================] - 0s 17ms/step


array([[3.1304855e-05]], dtype=float32)

Negative - OK

In [26]:
# Tokenize and vectorize it
vec_list = tokenize_and_vectorize([(1, test4)]) # Feed a sample to the tokenizer and vectorizer

# Convert the input to a fixed input length by padding or truncating
test_vec_list = pad_trunc(vec_list, maxlen) # convert the input to a fixed input length

# Input it to the model to predict the sentiment for it
test_vec = np.reshape(test_vec_list, (len(test_vec_list), maxlen, embedding_dims))

# Present the prediction
model_cnn1.predict(test_vec) # predict the sentiment of the input

1/1 [==============================] - 0s 17ms/step


array([[1.6613036e-05]], dtype=float32)

Negative - OK

In [27]:
# Tokenize and vectorize it
vec_list = tokenize_and_vectorize([(1, test5)]) # Feed a sample to the tokenizer and vectorizer

# Convert the input to a fixed input length by padding or truncating
test_vec_list = pad_trunc(vec_list, maxlen) # convert the input to a fixed input length

# Input it to the model to predict the sentiment for it
test_vec = np.reshape(test_vec_list, (len(test_vec_list), maxlen, embedding_dims))

# Present the prediction
model_cnn1.predict(test_vec) # predict the sentiment of the input

1/1 [==============================] - 0s 17ms/step


array([[2.9896473e-06]], dtype=float32)

Negative - OK

In [28]:
# Tokenize and vectorize it
vec_list = tokenize_and_vectorize([(1, test6)]) # Feed a sample to the tokenizer and vectorizer

# Convert the input to a fixed input length by padding or truncating
test_vec_list = pad_trunc(vec_list, maxlen) # convert the input to a fixed input length

# Input it to the model to predict the sentiment for it
test_vec = np.reshape(test_vec_list, (len(test_vec_list), maxlen, embedding_dims))

# Present the prediction
model_cnn1.predict(test_vec) # predict the sentiment of the input

1/1 [==============================] - 0s 18ms/step


array([[1.6771414e-06]], dtype=float32)

Positive - NO

In [29]:
# Tokenize and vectorize it
vec_list = tokenize_and_vectorize([(1, test7)]) # Feed a sample to the tokenizer and vectorizer

# Convert the input to a fixed input length by padding or truncating
test_vec_list = pad_trunc(vec_list, maxlen) # convert the input to a fixed input length

# Input it to the model to predict the sentiment for it
test_vec = np.reshape(test_vec_list, (len(test_vec_list), maxlen, embedding_dims))

# Present the prediction
model_cnn1.predict(test_vec) # predict the sentiment of the input

1/1 [==============================] - 0s 16ms/step


array([[8.467729e-07]], dtype=float32)

Test 1 only returns negative values. It also doesn't seem like positive reviews are closer to 1 than negative reviews.